In [1]:
import pandas as pd

df = pd.read_csv("../data/processed/ethiopia_fi_enriched.csv")


In [2]:
events = df[df['record_type'] == 'event']
impacts = df[df['record_type'] == 'impact_link']
observations = df[df['record_type'] == 'observation']


# Join Events to Impact Links

In [3]:
impact_model = impacts.merge(
    events,
    left_on='parent_id',
    right_on='event_name',
    how='left'
)

impact_model.head()


,record_id_x,record_type_x,category_x,pillar_x,indicator_x,indicator_code_x,indicator_direction_x,value_numeric_x,value_text_x,value_type_x,...,lag_months_y,evidence_basis_y,comparable_country_y,collected_by_y,collection_date_y,original_text_y,notes_y,event_name_y,event_date_y,parent_id_y
0,NaN,impact_link,NaN,access,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Digital ID simplifies KYC requirements,Fayda Digital ID National Rollout,2023-01-01,NaN


# Build Event–Indicator Association Matrix

In [8]:
# Filter only the rows that have impact_magnitude values
impact_model = df[df['record_type'] == 'impact_link'].copy()


In [9]:
impact_model['impact_magnitude'].value_counts(dropna=False)


impact_magnitude
medium    1
Name: count, dtype: int64

In [10]:
impact_model['impact_magnitude'] = impact_model['impact_magnitude'].fillna("No Impact")


In [11]:
association_matrix = impact_model.pivot(
    index='parent_id',                 # Each event
    columns='related_indicator',       # Each indicator
    values='impact_magnitude'          # The magnitude
)

# Fill any remaining missing values
association_matrix = association_matrix.fillna("No Impact")

# Optional: sort rows and columns for readability
association_matrix = association_matrix.sort_index().sort_index(axis=1)

print(association_matrix)




related_indicator                 ACC_OWNERSHIP
parent_id                                      
Fayda Digital ID National Rollout        medium
